## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,50.11,66,0,23.02,clear sky
1,1,Atuona,PF,-9.8000,-139.0333,78.06,77,81,18.63,light rain
2,2,Nanortalik,GL,60.1432,-45.2371,36.05,86,97,5.88,overcast clouds
3,3,Aksum,ET,14.1245,38.7244,58.17,36,24,4.43,few clouds
4,4,Coquimbo,CL,-29.9533,-71.3436,64.13,63,0,14.97,clear sky


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
#70-90

In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 90)]
# city_data_df.loc[city_data_df["Max Temp"] <= 90]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Atuona,PF,-9.8000,-139.0333,78.06,77,81,18.63,light rain
5,5,Tezu,IN,27.9167,96.1667,73.65,81,51,4.29,broken clouds
6,6,Young,UY,-32.6833,-57.6333,72.23,47,1,7.40,clear sky
7,7,Puerto Ayora,EC,-0.7393,-90.3518,78.76,84,96,7.00,light rain
8,8,Butaritari,KI,3.0707,172.7902,81.41,73,12,12.12,few clouds


In [18]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                299
City                   299
Country                297
Lat                    299
Lng                    299
Max Temp               299
Humidity               299
Cloudiness             299
Wind Speed             299
Current Description    299
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,78.06,light rain,-9.8000,-139.0333,
5,Tezu,IN,73.65,broken clouds,27.9167,96.1667,
6,Young,UY,72.23,clear sky,-32.6833,-57.6333,
7,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,
8,Butaritari,KI,81.41,few clouds,3.0707,172.7902,
9,Sao Filipe,CV,78.73,clear sky,14.8961,-24.4956,
12,Saint George,US,79.27,clear sky,37.1041,-113.5841,
13,Souillac,MU,73.36,overcast clouds,-20.5167,57.5167,
14,Hithadhoo,MV,81.70,broken clouds,-0.6000,73.0833,
23,Faanui,PF,80.11,light rain,-16.4833,-151.7500,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
# 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")
        

In [33]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()


City                   297
Country                297
Max Temp               297
Current Description    297
Lat                    297
Lng                    297
Hotel Name             297
dtype: int64

In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [37]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))